# The Cliff problem
Consider the following problem. An agent must navigate the grid world represented in Fig. 1, where the grey area corresponds to a cliff that the agent must avoid. The goal state corresponds to the cell marked with a G, while the cell marked with an S corresponds to a starting state. At every step, the agent receives a reward of -1 except in the cliff region, where the reward is -100. Whenever the agent steps into a cliff state, its position is reset back to the start state. When the agent steps into the goal state, the episode ends. The agent has available four actions: up (U), down (D), left (L) and right (R), all of which move the agent deterministically in the corresponding direction.

In this question, you will compare the performance of SARSA and Q-learning on the cliff task. To do so, assume that the agent follows an epsilon-greedy policy, with epsilon = 0.15. Run both algorithms for 500 episodes, making sure that the Q-values for both methods are initialized to 0. Consider throughout that gamma = 1 and use a step-size of alpha = 0.5.

## Question 1.
Compare:
* The total reward in each episode for Q-learning and SARSA, plotting the two in a single plot.
* The resulting policy after the 500 episodes.
Comment any differences observed.
**Note:** To mitigate the effect of noise on the plot, perform multiple runs and average the result across runs.

In Question 1 you implemented Q-learning and SARSA in a simple grid world domain, where exact representations for q* were possible. However, many domains are not amenable to such exact representations, due to the large size of the corresponding state spaces. In such domains, some form of function approximation is necessary. In the remainder of the homework, you will look at the problem of function approximation in RL.

In [ ]:
import numpy as np
from abc import ABC, abstractmethod
import matplotlib.pyplot as plt
import tqdm

In [ ]:
RUNS = 50 # Mitigate the effect of noise by averaging over x runs
EPISODES = 500
#EPSILON = 0.15
EPSILON = 0.1
GAMMA = 1
ALPHA = 0.5
REWARD = -1
REWARD_CLIFF = -100

WIDTH = 12
HEIGHT = 4

ACTIONS = ('up', 'down', 'left', 'right')

In [ ]:
class State:
    x: int
    y: int
    action_idx: int
    q_value: int

    def __init__(self, x: int, y: int, action_idx: int, q_value: int):
        self.x = x
        self.y = y
        self.action_idx = action_idx
        self.q_value = q_value

    def __repr__(self):
        return f'({self.x}, {self.y}, {ACTIONS[self.action_idx]}, {self.q_value})'

In [ ]:
class Environment:
    width: int
    height: int
    done: bool # whether the agent has arrived the goal G
    states: list[State] # required to update the rewards at the end of each episode
    
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.reset()

    def reset(self):
        self.x = 0
        self.y = self.height - 1
        self.done = False
        self.states = []
        return self.x, self.y
    
    def _is_cliff(self, x: int, y: int) -> bool:
        #return y == self.height - 1 and x != self.width - 1 and x != 0
        return y == self.height - 1 and 0 < x < self.width - 1
    
    def _is_goal(self, x: int, y: int) -> bool:
        #return y == self.height - 1 and x == self.width - 1
        return y == self.height - 1 and x == self.width - 1

    def step(self, action, action_idx) -> int:
        if action == 'down':
            self.y = min(self.y + 1, self.height - 1)
        elif action == 'right':
            self.x = min(self.x + 1, self.width - 1)
        elif action == 'up':
            self.y = max(self.y - 1, 0)
        elif action == 'left':
            self.x = max(self.x - 1, 0)
        else:
            raise ValueError('Invalid action')
        
        reward = 0

        if self._is_cliff(self.x, self.y):
            print("Cliff!")
            #self.reset()
            self.done = True
            reward = REWARD_CLIFF
        elif self._is_goal(self.x, self.y):
            print("Goal!")
            self.done = True
            reward = 0
        else:
            reward = REWARD
        
        state = State(self.x, self.y, action_idx, reward)
        self.states.append(state)
        
        return state 
        
    def render(self):
        for i in range(self.height):
            for j in range(self.width):
                # agent
                if i == self.y and j == self.x:
                    print('A', end='')
                # goal
                elif self._is_goal(j, i):
                    print('G', end='')
                # cliff 
                elif self._is_cliff(j, i):
                    print('C', end='')
                else:
                    print('X', end='')
            print()
        print()

In [ ]:
class Agent(ABC):
    epsilon: float
    gamma: float
    alpha: float
    actions: list[str]
    env: Environment
    q_table: np.ndarray

    def __init__(self, epsilon, gamma, alpha, actions: list[str], env: Environment):
        self.epsilon = epsilon
        self.gamma = gamma
        self.alpha = alpha
        self.actions = actions
        self.env = env
        self.q_table = np.zeros((self.env.width, self.env.height, len(self.actions)))

    def reset(self):
        self.q_table = np.zeros((self.env.width, self.env.height, len(self.actions)))

    def _choose_action(self) -> (str, int):
        """
        Choose an action based on epsilon-greedy policy.
        """
        action_idx = 0
        # Explore
        if np.random.uniform(0, 1) <= self.epsilon:
            action_idx = np.random.choice(range(len(self.actions)))
        # Exploit
        else:
            action_idx = np.argmax(self.q_table[self.env.x, self.env.y])
        action = self.actions[action_idx]
        return action, action_idx
    
    @abstractmethod
    def _update_rewards(self, 
                        current_x: int, 
                        current_y: int, 
                        current_action_idx: int, 
                        reward, next_x: int, 
                        next_y: int, 
                        next_action_idx: int) -> (int, list[str]):
        """
        Update the rewards for the current state-action pair.
        """
        pass

    def play_episode(self) -> int:
        current_x, current_y = self.env.x, self.env.y
        current_action, current_action_idx = self._choose_action()
        total_reward = 0
        while self.env.done == False:
            next_state = self.env.step(current_action, current_action_idx)
            print(f"State: {next_state}")
            next_x, next_y = next_state.x, next_state.y
            reward = next_state.q_value
            total_reward += reward
            next_action, next_action_idx = self._choose_action()
            self._update_rewards(current_x, current_y, current_action_idx, reward, next_x, next_y, next_action_idx)
            current_x, current_y, current_action, current_action_idx = next_x, next_y, next_action, next_action_idx
            #self.env.render()
        #self.env.reset()
        return total_reward
    
    def print_policy(self) -> int:
        print("---> Agent's policy:")
        current_action_idx = np.argmax(self.q_table[self.env.x, self.env.y])
        current_action = self.actions[current_action_idx]
        total_reward = 0
        while self.env.done == False:
            next_state = self.env.step(current_action, current_action_idx)
            print(f"State: {next_state}")
            reward = next_state.q_value
            total_reward += reward
            next_action_idx = np.argmax(self.q_table[self.env.x, self.env.y])
            next_action = self.actions[next_action_idx]
            current_action, current_action_idx = next_action, next_action_idx
        return total_reward, self.env.states

class SarsaAgent(Agent):
    def _update_rewards(self, current_x: int, 
                        current_y: int, 
                        current_action_idx: int, 
                        reward, next_x: int, 
                        next_y: int, 
                        next_action_idx: int):
        current_q = self.q_table[current_x, current_y, current_action_idx]
        next_q = self.q_table[next_x, next_y, next_action_idx]
        target = reward + self.gamma * next_q
        self.q_table[current_x, current_y, current_action_idx] += self.alpha * (target - current_q)

class QLearningAgent(Agent):
    def _update_rewards(self, 
                        current_x: int, 
                        current_y: int, 
                        current_action_idx: int, 
                        reward, next_x: int, 
                        next_y: int, 
                        next_action_idx: int):
        current_q = self.q_table[current_x, current_y, current_action_idx]
        next_max_q = np.max(self.q_table[next_x, next_y])
        target = reward + self.gamma * next_max_q
        self.q_table[current_x, current_y, current_action_idx] += self.alpha * (target - current_q)

In [ ]:
class Game:
    episodes: int
    agent: Agent

    def __init__(self, episodes: int, agent: Agent) -> None:
        self.episodes = episodes
        self.agent = agent

    def run(self) -> list[int]:
        total_rewards = []
        for i in range(self.episodes):
            print(f"\n---------- EPISODE {i} ----------")
            total_reward = self.agent.play_episode()
            #self.agent.reset()
            self.agent.env.reset()
            total_rewards.append(total_reward)
        # prints set of actions performed in the last episode
        return total_rewards

In [ ]:
total_rewards_sarsa = np.zeros((RUNS, EPISODES))
for run in tqdm.tqdm(range(RUNS)):    
    sarsa_env = Environment(WIDTH, HEIGHT)
    #sarsa_env.render()
    sarsa_agent = SarsaAgent(EPSILON, GAMMA, ALPHA, ACTIONS, sarsa_env)
    sarsa_game = Game(EPISODES, sarsa_agent)
    total_rewards_sarsa[run] = sarsa_game.run()
    if run == RUNS-1:
        sarsa_agent.print_policy()
#print(f"total_rewards_sarsa: {total_rewards_sarsa}")

In [ ]:
total_rewards_q_learning = np.zeros((RUNS, EPISODES))
for run in range(RUNS):     
    qlearning_env = Environment(WIDTH, HEIGHT)
    #qlearning_env.render()
    qlearning_agent = QLearningAgent(EPSILON, GAMMA, ALPHA, ACTIONS, qlearning_env)
    qlearning_game = Game(EPISODES, qlearning_agent)
    total_rewards_q_learning[run] = qlearning_game.run()
    if run == RUNS-1:
        qlearning_agent.print_policy()
#print(f"total_rewards_q_learning: {total_rewards_q_learning}")


In [ ]:
def plot_rewards_sum_per_episode_comp(sarsa_rewards, qlearning_rewards):
    plt.plot(range(EPISODES), sarsa_rewards, label='Sarsa')
    plt.plot(range(EPISODES), qlearning_rewards, label='Q-Learning')
    plt.xlabel('Episodes')
    plt.ylabel('Sum of rewards during episode')
    plt.ylim([-150, 0])
    plt.legend()
    plt.show()

In [ ]:
avg_rewards_sarsa = np.mean(total_rewards_sarsa, axis=0)
avg_rewards_q_learning = np.mean(total_rewards_q_learning, axis=0)
plot_rewards_sum_per_episode_comp(avg_rewards_sarsa, avg_rewards_q_learning)

### Interpretation

The final policies obtained were:
* SARSA: 
    State: (0, 2, up, -1)
    State: (0, 1, up, -1)
    State: (1, 1, right, -1)
    State: (1, 0, up, -1)
    State: (2, 0, right, -1)
    State: (3, 0, right, -1)
    State: (4, 0, right, -1)
    State: (5, 0, right, -1)
    State: (6, 0, right, -1)
    State: (7, 0, right, -1)
    State: (8, 0, right, -1)
    State: (9, 0, right, -1)
    State: (10, 0, right, -1)
    State: (11, 0, right, -1)
    State: (11, 1, down, -1)
    State: (11, 2, down, -1)
    Goal!
    State: (11, 3, down, 0)
* Q-learning:
    State: (0, 2, up, -1)
    State: (1, 2, right, -1)
    State: (2, 2, right, -1)
    State: (3, 2, right, -1)
    State: (4, 2, right, -1)
    State: (5, 2, right, -1)
    State: (6, 2, right, -1)
    State: (7, 2, right, -1)
    State: (8, 2, right, -1)
    State: (9, 2, right, -1)
    State: (10, 2, right, -1)
    State: (11, 2, right, -1)
    Goal!
    State: (11, 3, down, 0)

To obtain this plot, I averaged the sum of rewards across 50 different runs. In the first episodes, both SARSA and Q-learning are making several suboptimal decisions, obtaining very negative sums of rewards between -110 and -140. As the number of episodes increases, the sums of rewards for both algorithms increase very fast, stabilizing around after 100 episodes. After these 100 episodes, we can observe the trend that most distinguishes both algorithms for this scenario: SARSA receives slightly higher sums of rewards and has less variability than Q-learning. This is due to SARSA, despite not achieving the optimal policy, being an on-policy algorithm, it uses the next action it takes to update its q-values. This way, it learns that going close to the edge of the cliff can be costly (even after updating the q-values for several episodes, it still performs exploratory actions due to the epsilon greedy policy) and stays away from the cliff. Q-learning, on the other hand, is an off-policy method, meaning that it does not use the policy used to choose the next action to update the q-values. It uses a slightly different policy, in which it chooses the action with highest q-value and does not take into account the epsilon-exploration, so it does not think that going to the edge is costly. This allows it to learn the optimal policy, but it ends up performing worse in terms of sums of rewards because the agent will end up falling in the cliff more easily, since all it takes is an exploratory action to fall into the cliff.